In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('hexoskin_2bands_training_set_10_final.xlsx')
df

,exp_diff_mean,A_exp_dur_sd,A_exp_diff_mean,exp_dur_sd,duty_cycle_mean,A_RRV_RMSSD,RRV_RMSSD,ie_ratio_mean,A_duty_cycle_mean,insp_flow_sd,Task_Label,Participant,Classification
0,0.127790,0.164364,0.117188,0.156604,0.492097,404.771735,376.386712,0.971761,0.492399,1.445912e+06,12.0a,10785,0
1,0.201451,0.174588,0.241071,0.152211,0.486992,392.384431,351.137516,0.950939,0.485336,1.034691e+06,12.0b,10785,0
2,0.199219,0.251690,0.236719,0.213642,0.476697,459.428807,457.608476,0.913378,0.482721,1.890795e+06,12.0c,10785,0
3,0.285156,0.197705,0.266276,0.213173,0.477750,450.569753,439.827676,0.918119,0.484587,1.316481e+06,12.0d,10785,0
4,0.159375,0.193377,0.108073,0.213828,0.481671,516.192869,457.645154,0.936229,0.488034,9.634884e+05,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,0.203125,0.209238,0.218316,0.200555,0.490259,357.971103,334.175635,0.967894,0.488741,2.209139e+06,85,98586,0
2900,0.921387,0.731201,0.941894,0.725393,0.401725,1042.040237,1209.145284,0.933997,0.431877,1.280213e+07,87,98586,1
2901,0.106337,0.115003,0.101562,0.117894,0.496388,153.331025,185.358320,0.987026,0.494804,2.990174e+06,89,98586,0
2902,2.145182,1.115591,1.260045,1.447558,0.412197,2387.940073,3053.223554,0.748645,0.360902,1.441962e+07,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.27790200e-01, 1.64364500e-01, 1.17187500e-01, ...,
         9.71761000e-01, 4.92398700e-01, 1.44591181e+06],
        [2.01450900e-01, 1.74588400e-01, 2.41071400e-01, ...,
         9.50939100e-01, 4.85336100e-01, 1.03469140e+06],
        [1.99218800e-01, 2.51689700e-01, 2.36718800e-01, ...,
         9.13377900e-01, 4.82720700e-01, 1.89079541e+06],
        ...,
        [1.06336800e-01, 1.15003100e-01, 1.01562500e-01, ...,
         9.87025600e-01, 4.94804200e-01, 2.99017395e+06],
        [2.14518230e+00, 1.11559080e+00, 1.26004460e+00, ...,
         7.48644500e-01, 3.60902200e-01, 1.44196160e+07],
        [1.23535200e-01, 1.63306400e-01, 1.32812500e-01, ...,
         9.26293000e-01, 4.83311700e-01, 2.04013979e+06]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['exp_diff_mean',
  'A_exp_dur_sd',
  'A_exp_diff_mean',
  'exp_dur_sd',
  'duty_cycle_mean',
  'A_RRV_RMSSD',
  'RRV_RMSSD',
  'ie_ratio_mean',
  'A_duty_cycle_mean',
  'insp_flow_sd'],
 'partici

In [4]:
#running the nested cross-validation for the Accelerometer method using Random Forest
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

rfc = RandomForestClassifier(random_state=random_seed, class_weight="balanced")

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.98484848 0.90909091 0.92424242 0.81818182 0.98484848 0.96969697
 0.98484848 0.84848485 0.92424242 0.92424242 0.89393939 0.63636364
 0.66666667 0.83333333 0.83333333 0.90909091 1.         0.78787879
 0.93939394 0.86363636 0.84848485 0.95454545 0.77272727 0.90909091
 0.86363636 0.95454545 0.89393939 0.98484848 0.74242424 0.77272727
 0.86363636 0.92424242 0.84848485 0.95454545 0.83333333 0.8030303
 0.93939394 0.89393939 0.96969697 0.93939394 0.81818182 0.8030303
 0.66666667 0.92424242]
mean score:            0.8753
nested_precision_scores:    [0.98520085 0.91264822 0.92611833 0.86246658 0.98520085 0.97107438
 0.98545455 0.8790107  0.92532151 0.93730408 0.91788856 0.72027972
 0.77380952 0.83512195 0.83215369 0.91831852 1.         0.78458498
 0.93939394 0.87782767 0.89304813 0.95459141 0.86013986 0.90909091
 0.90082645 0.95757576 0.9000259  0.98545455 0.79296188 0.77498891
 0.88797654 0.93730408 0.89304813 0.95959596 0.85806452 0.87223587
 0.94008264 0.89337255

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 2:
Sensitivity: 0.7917
Specificity: 0.9762
Fold 3:
Sensitivity: 0.8333
Specificity: 0.9762
Fold 4:
Sensitivity: 0.9583
Specificity: 0.7381
Fold 5:
Sensitivity: 0.9583
Specificity: 1.0000
Fold 6:
Sensitivity: 0.9167
Specificity: 1.0000
Fold 7:
Sensitivity: 1.0000
Specificity: 0.9762
Fold 8:
Sensitivity: 0.9583
Specificity: 0.7857
Fold 9:
Sensitivity: 0.9167
Specificity: 0.9286
Fold 10:
Sensitivity: 1.0000
Specificity: 0.8810
Fold 11:
Sensitivity: 1.0000
Specificity: 0.8333
Fold 12:
Sensitivity: 0.8333
Specificity: 0.5238
Fold 13:
Sensitivity: 0.9167
Specificity: 0.5238
Fold 14:
Sensitivity: 0.7917
Specificity: 0.8571
Fold 15:
Sensitivity: 0.7500
Specificity: 0.8810
Fold 16:
Sensitivity: 0.9583
Specificity: 0.8810
Fold 17:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 18:
Sensitivity: 0.6250
Specificity: 0.8810
Fold 19:
Sensitivity: 0.9167
Specificity: 0.9524
Fold 20:
Sensitivity: 0.9167
Specificity: 0.8333
Fold 21:
Sensitivity: 1.0000


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8753443526170798, SD=0.08948041328006305, 95% CI=(0.8489050659270876, 0.901783639307072), Min=0.6363636363636364, Max=1.0
test_precision: Mean=0.8969311390891694, SD=0.06671325870105009, 95% CI=(0.8772189930656589, 0.91664328511268), Min=0.7202797202797203, Max=1.0
test_recall: Mean=0.8753443526170798, SD=0.08948041328006305, 95% CI=(0.8489050659270876, 0.901783639307072), Min=0.6363636363636364, Max=1.0
test_f1_score: Mean=0.8704596829614615, SD=0.09012922765895849, 95% CI=(0.8438286873963827, 0.8970906785265402), Min=0.6360294117647058, Max=1.0
test_roc_auc: Mean=0.9529671717171719, SD=0.05423259854366268, 95% CI=(0.9369427573422483, 0.9689915860920956), Min=0.7475198412698413, Max=1.0
sensitivity_scores: Mean=0.9185606060606063, SD=0.08756436607038791, 95% CI=(0.8926874647384233, 0.9444337473827893), Min=0.625, Max=1.0
specificity_scores: Mean=0.8506493506493507, SD=0.13383898309013323, 95% CI=(0.8111031850193191, 0.8901955162793822), Min=0.5, Max=1.0


In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 2 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 3 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 4 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 5 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 6 - Best hyperparameters: {'max_depth': 9, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 7 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 8 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 9 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: max_depth - Most frequent value: 6 (Count: 39)
Hyperparameter: max_features - Most frequent value: None (Count: 44)
Hyperparameter: max_leaf_nodes - Most frequent value: 9 (Count: 44)
Hyperparameter: n_estimators - Most frequent value: 25 (Count: 16)
